In [8]:
import os
os.chdir(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\SPFD')
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer
from sklearn.decomposition import PCA, TruncatedSVD

from estimators import (AverageKLPE, MaxKLPE, OCSVM,
                                       KernelSmoothing, IsolationForest, LocalOutlierFactor)
from tuning import anomaly_tuning
import time

def fillAnomaly(req_data_ref, df, scale_param, time_df, ref):
    temp1 = df.copy()
    n_estimator = 50
    N_JOBS = 1
    n_samples = df.shape[0]
    alpha_set = 0.9

    X = np.asarray(df[:])

    #algorithms = [AverageKLPE, MaxKLPE, OCSVM, IsolationForest, LocalOutlierFactor]
    #algorithms = [AverageKLPE]
    algorithms = [IsolationForest]
    #algorithms = [OCSVM]
    #algorithms = [LocalOutlierFactor]
    scaling_techniques = {'StandardScaler' :StandardScaler, 'MinMaxScaler':MinMaxScaler, 'MaxAbsScaler':MaxAbsScaler, 'Normalizer':Normalizer}
    algo_param = {'aklpe': {'k': np.arange(1, min(50, int(0.8 * n_samples)), 2),
                            'novelty': [True]},
                  'mklpe': {'k': np.arange(1, min(50, int(0.8 * n_samples)), 2),
                            'novelty': [True]},
                  'ocsvm': {'sigma': np.linspace(0.01, 5., 50)},
                  'iforest': {'max_samples': np.linspace(0.1, 1., 10)},
                  'ks': {'bandwidth': np.linspace(0.01, 5., 50)},
                  'LOF':{'n_neighbors': np.arange(1, min(50, int(0.8 * n_samples)), 2)}
                  }

    
    if scale_param :
        print('Preparocssing data with ' + scale_param)
        X = scaling_techniques[scale_param]().fit_transform(X)

    
    # Tuning step
    cv = ShuffleSplit(n_splits=n_estimator, test_size=0.2, random_state=42)
    

    
    outliers = {}
    for algo in algorithms:
        

        name_algo = algo.name
        print('--------------', name_algo, ' -------------')
        parameters = algo_param[name_algo]
        dat_length = len(X)
        start_time = time.ctime()

        models, off_set = anomaly_tuning(X, base_estimator=algo,
                                   parameters=parameters,
                                   random_state=42,
                                   cv=cv, n_jobs=N_JOBS)
        
        end_time = time.ctime()
        temp_dict = {"Data_length" :[dat_length], "Algo_name":[algo.name], "start_time":start_time, "end_time":end_time }
        temp_df =pd.DataFrame.from_dict(temp_dict) 
        time_df = pd.concat([time_df, temp_df], ignore_index =True)

        Z_data = np.zeros((n_samples,))
        
        if algo.name == 'LOF':
            for n_est in range(n_estimator):
                clf = models[n_est]
                print("Z_data_shape", Z_data.shape, "clf score sample", (clf.score_samples(X)).shape)
                Z_data += 1. / n_estimator * (clf.score_samples(X))

            off_data = np.percentile(Z_data, 100 * (1 - alpha_set))
            outliers[name_algo] = Z_data - off_data < 0

            temp =  pd.DataFrame(outliers)
            temp1 = pd.concat([req_data_ref, temp1], axis=1)
            cln = algo.name + '_scores'
            temp1[cln] = Z_data
            (pd.concat([temp1,temp], axis =1)).to_excel(name_algo+ref+'_results.xlsx', index = False)
        else:
            
            for n_est in range(n_estimator):
                clf = models[n_est]
                Z_data += 1. / n_estimator * (clf.score_samples(X))
            off_data = np.percentile(Z_data, 100 * (1 - alpha_set))
            outliers[name_algo] = Z_data - off_data < 0
            temp =  pd.DataFrame(outliers)
            temp1 = pd.concat([req_data_ref, temp1], axis=1)
            cln = algo.name + '_scores'
            temp1[cln] = Z_data
            (pd.concat([temp1, temp], axis =1)).to_excel(name_algo+ref+'_results.xlsx', index = False)
            

            
    outliers = pd.DataFrame(outliers)
    return outliers, time_df, clf, off_set, Z_data

In [9]:
res_dict = dict()
refer = '_0703_iforest'
df = pd.read_pickle('OMD_0703.pkl')
res, time, models, off_set, scores = fillAnomaly(req_data_ref = df[['AGENT1','YEAR']],
                        df=df.drop(['AGENT1','YEAR'],axis=1),
                       scale_param = 'StandardScaler',
                       time_df = pd.DataFrame(columns = ["Data_length", "Algo_name", "start_time", "end_time"]),
                       ref = refer)

res_dict[refer] = [res, time, models, off_set, scores]

Preparocssing data with StandardScaler
-------------- iforest  -------------
Dimension might be too high for volume estimation and thus deteriorate model selection
running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out o

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0}
9 iterations completed out of 10 for  iforest


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.5min remaining:    0.0s


running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0}
9 iterations completed out of 10 for  iforest


...

8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0}
9 iterations completed out of 10 for  iforest
running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0

7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0}
9 iterations completed out of 10 for  iforest
running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.9

6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0}
9 iterations completed out of 10 for  iforest
running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.8

5 iterations completed out of 10 for  iforest
params {'max_samples': 0.70000000000000007}
6 iterations completed out of 10 for  iforest
params {'max_samples': 0.80000000000000004}
7 iterations completed out of 10 for  iforest
params {'max_samples': 0.90000000000000002}
8 iterations completed out of 10 for  iforest
params {'max_samples': 1.0}
9 iterations completed out of 10 for  iforest
running base_estimator <class 'estimators.IsolationForest'>
params {'max_samples': 0.10000000000000001}
0 iterations completed out of 10 for  iforest
params {'max_samples': 0.20000000000000001}
1 iterations completed out of 10 for  iforest
params {'max_samples': 0.30000000000000004}
2 iterations completed out of 10 for  iforest
params {'max_samples': 0.40000000000000002}
3 iterations completed out of 10 for  iforest
params {'max_samples': 0.5}
4 iterations completed out of 10 for  iforest
params {'max_samples': 0.59999999999999998}
5 iterations completed out of 10 for  iforest
params {'max_samples': 0.7

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 89.5min finished
